### **Introduction**


**A model is, abstractly:**

* A function that computes something on tensors (a forward pass)
* Some variables that can be updated in response to training

### **Setup**


In [29]:
import tensorflow as tf
from datetime import datetime
import random as ra
import tensorboard as tb

print(tb.__version__)
print(tf.__version__)

2.7.0
2.6.0


### **Defining Models and Layers in TensorFlow**
**Layers are functions with a known `mathematical structure` that can be reused and have `trainable variables`.**

In [30]:
# Creating a simple Model using tf.Module
class Module(tf.Module):
    def __init__(self, name):
        super().__init__(name= name)
        self.tr_var = tf.Variable(5.0, name= 'train_me')
        self.non_tr_var = tf.Variable(5.0, trainable=False , name= 'not_train')

    def __call__(self, x):
        return self.tr_var * x + self.non_tr_var

model = Module('SimpleModel')
model(tf.constant(3.)).numpy()

20.0

In [31]:
# Printing trainable & non_trainable variables
print('Trainable variables: ', model.trainable_variables)
print('Non-Trainable variables', model.non_trainable_variables)
print()
print('All variables: ')
for var in model.variables:
    print(var)

Trainable variables:  (<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>,)
Non-Trainable variables (<tf.Variable 'not_train:0' shape=() dtype=float32, numpy=5.0>,)

All variables: 
<tf.Variable 'not_train:0' shape=() dtype=float32, numpy=5.0>
<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>


> **Creating a two-layer linear layer model made out of modules.**

In [32]:
# Creating a dense layer
class Dense(tf.Module):
    def __init__(self, in_fea, out_fea,name=None):
        super().__init__(name)
        self.wei = tf.Variable(
            tf.random.normal([in_fea, out_fea], name= 'weights')
        )
        self.bias = tf.Variable(
            tf.zeros([out_fea], name='bias')
        )

    def __call__(self, x_ten):
        # Computation: Y = Matrix multiplication (x & weights) plus the model bias
        # Computation: tf.nn.relu(Y)
        y = (x_ten @ self.wei) + self.bias
        return tf.nn.relu(y)

In [33]:
class Sequential(tf.Module):
    def __init__(self, name=None):
        super().__init__(name)
        
        self.dense1 = Dense(3, 3, 'InputLayer')
        self.dense2 = Dense(3, 2, 'OutputLayer')
        
    def __call__(self, tx):
        y = self.dense1(tx)
        return self.dense2(y)

my_model = Sequential('AI_Model')
# Model acceptable shape [n x 3]
my_model(tf.random.poisson([2, 3], 5.5)) 
my_model(tf.random.normal([3, 3], mean=17, stddev=3.5)).numpy()


array([[ 0.      , 30.625645],
       [ 0.      , 39.363396],
       [ 0.      , 34.99908 ]], dtype=float32)

In [34]:
# Printing Model submodules
for mod in my_model.submodules:
    print(mod)
print()

# Printing model Variables
for var in  my_model.variables: 
    print(f'var: {var.numpy()}\n')




var: [0. 0. 0.]

var: [[ 0.959555    0.0199919   1.7453369 ]
 [-0.15266621 -0.33506396 -0.18785717]
 [ 0.88807493 -0.5826032   0.74195546]]

var: [0. 0.]

var: [[ 0.06880555  0.58556736]
 [ 1.7550813   0.97906375]
 [-0.55982     0.4991804 ]]



* ##### **Waiting to create variables**


In [35]:
# The Input shape of the layers is dynamically inferered at the runtime by the size of the incoming tensor..
class dyn_Layer(tf.Module):
    def __init__(self, outCh, name=None):
        super().__init__(name)
        self.is_built = False 
        self.out_channels = outCh
    
    @tf.function
    def __call__(self, tx):
        if not self.is_built:
            self.is_built = True
            self.wei = tf.Variable(
                tf.random.normal([tx.shape[-1], self.out_channels]), name='Weights'
            )
            self.bias = tf.Variable(
                tf.zeros([self.out_channels], name='bias')
            )
        
        y = tx @ self.wei + self.bias
        return tf.nn.relu(y)
    
class dyn_SeqModel(tf.Module):
    def __init__(self, name=None):
        super().__init__(name)
        self.dense1 = dyn_Layer(12, 'Input_Layer')
        self.dense2 = dyn_Layer(3, 'Output_Layer')
        
    @tf.function
    def __call__(self, tx):
        tx = self.dense1(tx)
        return self.dense2(tx)

In [36]:
# Calling the dynamic model on different sets of inputs
for i in range(10):
    in_chs = ra.randint(1,15)
    print('InChanels for this Model', in_chs)
    model = dyn_SeqModel('AdaptiveModel')
    tx = tf.random.uniform([6, 6], minval=16, maxval=32)
    print(model(tx))

# Our model is working...



InChanels for this Model 11
tf.Tensor(
[[143.36325   42.863983   0.      ]
 [122.86427   52.69522    0.      ]
 [ 98.208435  63.0875     0.      ]
 [105.95747   30.056622   0.      ]
 [116.52232   31.957214   0.      ]
 [ 83.90777   11.676098   0.      ]], shape=(6, 3), dtype=float32)
InChanels for this Model 3
tf.Tensor(
[[  0.      154.6011    0.     ]
 [  0.      173.68495   0.     ]
 [  0.      153.67804   0.     ]
 [  0.      160.19254   0.     ]
 [  0.      162.41951   0.     ]
 [  0.      196.6761    0.     ]], shape=(6, 3), dtype=float32)
InChanels for this Model 6
tf.Tensor(
[[13.901243  0.       37.83688 ]
 [58.861008  0.       17.698965]
 [67.54575   0.        4.008301]
 [31.108435  0.       23.12328 ]
 [43.71979   0.       21.07631 ]
 [58.71685   0.       54.907845]], shape=(6, 3), dtype=float32)
InChanels for this Model 7
tf.Tensor(
[[  0.         0.        92.292336]
 [  0.         0.       105.3203  ]
 [  0.         0.        99.48811 ]
 [  0.         0.        95.66135 

### **Saving Weights**

In [37]:
# Create a dir to save weights
chk_path = 'Weights_checkpoint'
checkpoint = tf.train.Checkpoint(model=model)

# Using this Command, the weights are stored into the file.
checkpoint.write(chk_path)
tf.train.list_variables(chk_path)

# Creating a new Model and assigning the `SAME WEIGHTS`
new_mod = dyn_SeqModel('Dynamic_Seq2')
checkpoint2 = tf.train.Checkpoint(model=new_mod)

# Reloading the previous weights
checkpoint2.restore('Weights_checkpoint')

# Testing the new Model on previous input, should be able to produce same output
print(f'Previous Model results:\n {model(tx).numpy()}\n')
print(f'New Model result after restoring weights:\n {new_mod(tx).numpy()}')

print("It worked")


Previous Model results:
 [[  0.         49.09887     0.       ]
 [ 16.18663    38.172913   51.352745 ]
 [  0.         35.276817    0.       ]
 [  7.6772537  42.208687   41.35611  ]
 [  0.         60.66057     0.       ]
 [113.41975    57.09588     0.       ]]

New Model result after restoring weights:
 [[  0.         49.09887     0.       ]
 [ 16.18663    38.172913   51.352745 ]
 [  0.         35.276817    0.       ]
 [  7.6772537  42.208687   41.35611  ]
 [  0.         60.66057     0.       ]
 [113.41975    57.09588     0.       ]]
It worked


### **Saving Functions + Visualization using Tensorboard**

In [ ]:
# setting up logging
timest = datetime.now().strftime("%Y%m%d-%H%M%S")
logs_dir = 'logs/func/%s' % timest
writer = tf.summary.create_file_writer(logs_dir)

# Create new model to get fresh trace
SeqModel = dyn_SeqModel('Traced_Model')
tf.summary.trace_on(graph= True)
tf.profiler.experimental.start(logs_dir)

# Only call one tf.function while tracing
z = print(SeqModel(tf.random.uniform(shape=[4, 9], minval= 50, maxval= 70)))

with writer.as_default():
    tf.summary.trace_export(
        name= 'Function Trace',
        step= 0,
        profiler_outdir= logs_dir
    )

> **Starting Tensorboard to visualize tracing results**

In [39]:
%tensorboard --logs/func

# Masha-Allah it worked

UsageError: Line magic function `%tensorboard` not found.


* ##### **Creating a saved Model**


In [40]:
tf.saved_model.save(SeqModel, 'SavedModel')


INFO:tensorflow:Assets written to: SavedModel\assets


INFO:tensorflow:Assets written to: SavedModel\assets


### **Keras Models and Layers**

* ##### **Keras Layers**

* ##### **The build step**


* ##### **Keras Models**


### **Saving Keras Model**

### **`Sonnet`: Alternate deep-learning API for building ML Models**
> **By DeepMind,**
> **Based on tf.Module**